<a href="https://colab.research.google.com/github/Simu96/2020-2-OSSP1-WhatsUp-5/blob/sim/Data_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>